# 2-S4: 비용 최적화 + CI/CD 기초

FDS 비용 함수와 CI/CD 파이프라인을 학습합니다.

## 학습 목표
1. 왜 수동 배포가 위험한가?
2. CI/CD 기초 개념 (CI vs CD)
3. GitHub Actions 기초 (Workflow, Job, Step)
4. ML 프로젝트 테스트 (데이터 품질, 모델 성능, 추론 시간)
5. FDS 비용 함수 (FN/FP 비용)

## 예상 시간
- 약 1.5시간

## 선수 조건
- 2-S3 Evidently 드리프트 완료
- Python 기초

## 현업 표준 (2025)
- actions/checkout@**v5**
- actions/setup-python@**v5**
- actions/upload-artifact@**v4**

In [ ]:
# 필요한 라이브러리
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

---
## 1. 왜 수동 배포가 위험한가?

### 현업 사고 사례

```
금요일 오후 5시
"이번 모델 성능 좋아 보이네. 빨리 배포하고 퇴근하자!"

1. 로컬에서 테스트 안 함 ("어차피 잘 되겠지")
2. 서버에 직접 파일 복사 (git 없이)
3. 재시작 후 에러 발생!
4. 롤백하려니 이전 버전 어디 있는지 모름...
5. 주말 내내 장애 대응
```

### 수동 배포의 문제점

| 문제 | 설명 | 결과 |
|------|------|------|
| **휴먼 에러** | 복붙 실수, 설정 누락 | 서비스 장애 |
| **테스트 생략** | 급해서 테스트 건너뜀 | 버그 배포 |
| **버전 관리 없음** | 어떤 버전이 배포됐는지 모름 | 롤백 불가 |
| **환경 차이** | 로컬과 서버 환경 다름 | "내 컴에선 됐는데..." |

### CI/CD가 해결하는 것

```
자동화된 CI/CD 파이프라인

1. 코드 푸시 -> 자동 테스트 실행
2. 테스트 실패 -> 배포 차단! (안전장치)
3. 테스트 통과 -> 자동 배포
4. 문제 발생 -> 이전 버전으로 자동 롤백
5. 모든 이력 기록됨 -> 추적 가능
```

In [ ]:
# 체크포인트 1: CI/CD 필요성 이해

cicd_benefits = {
    "자동_테스트": True,      # 코드 변경 시 자동으로 테스트 실행
    "휴먼_에러_방지": True,   # 사람 실수 줄임
    "버전_추적": True,        # 어떤 버전이 배포됐는지 기록
    "빠른_롤백": True,        # 문제 시 이전 버전으로 빠르게 복귀
}

# 검증
assert all(cicd_benefits.values()), "모든 항목이 CI/CD의 장점입니다!"
print("체크포인트 1 통과!")

---
## 2. CI/CD 기초 개념

### CI (Continuous Integration) - 지속적 통합

```
CI = 코드 변경 시 자동 테스트

개발자 A: 코드 푸시 -+
                    +-> [자동 테스트] -> 결과 알림
개발자 B: 코드 푸시 -+

- 여러 개발자의 코드를 자주 통합
- 통합할 때마다 자동 테스트
- 버그를 빨리 발견 ("빨리 실패하라")
```

### CD (Continuous Deployment) - 지속적 배포

```
CD = 테스트 통과 시 자동 배포

[코드 푸시] -> [테스트 통과] -> [자동 배포] -> [프로덕션]
                    |
              [테스트 실패] -> 배포 차단!

- 테스트 통과한 코드만 배포
- 사람 개입 최소화
- 더 자주, 더 안전하게 배포
```

### 일반 소프트웨어 vs ML 프로젝트

| 구분 | 일반 소프트웨어 | ML 프로젝트 |
|------|----------------|-------------|
| **코드 변경** | 주요 트리거 | 트리거 중 하나 |
| **데이터 변경** | 거의 없음 | 중요한 트리거 |
| **모델 재학습** | N/A | 새로운 배포 |
| **테스트 항목** | 기능, 유닛 테스트 | + 성능, 드리프트 검사 |
| **롤백 대상** | 코드 | 코드 + 모델 |

In [ ]:
# 예제: CI vs CD 구분하기

ci_cd_examples = {
    "코드 푸시 시 pytest 자동 실행": "CI",
    "테스트 통과 후 서버에 자동 배포": "CD",
    "PR 생성 시 코드 리뷰 봇 실행": "CI",
    "main 브랜치 머지 시 프로덕션 반영": "CD",
    "일일 스케줄로 모델 재학습 후 배포": "CD",
    "모델 변경 시 성능 테스트 자동 실행": "CI",
}

for action, stage in ci_cd_examples.items():
    print(f"[{stage}] {action}")

In [ ]:
# 실습 1: CI vs CD 구분하기

quiz = {
    "GitHub에 푸시하면 자동으로 유닛 테스트 실행": "CI",  # 테스트 = CI
    "테스트 통과하면 Docker 이미지 빌드 후 배포": "CD",   # 배포 = CD  
    "데이터 품질 검사 실패 시 파이프라인 중단": "CI",     # 검사 = CI
}

# 체크포인트
assert quiz["GitHub에 푸시하면 자동으로 유닛 테스트 실행"] == "CI"
assert quiz["테스트 통과하면 Docker 이미지 빌드 후 배포"] == "CD"
assert quiz["데이터 품질 검사 실패 시 파이프라인 중단"] == "CI"
print("체크포인트 2 통과! CI는 테스트/검사, CD는 배포")

---
## 3. GitHub Actions 기초

### GitHub Actions란?

GitHub에서 제공하는 **무료** CI/CD 서비스입니다.

```
GitHub Actions

- GitHub 저장소에 코드 푸시 -> 자동으로 워크플로 실행
- YAML 파일로 파이프라인 정의
- 무료 (공개 저장소 무제한, 비공개도 월 2,000분 무료)
```

### 핵심 용어 5가지

| 용어 | 설명 | 비유 |
|------|------|------|
| **Workflow** | 전체 자동화 파이프라인 | 요리 레시피 전체 |
| **Job** | 워크플로 내 작업 단위 | 레시피의 각 단계 |
| **Step** | Job 내 개별 명령 | 단계별 세부 동작 |
| **Action** | 재사용 가능한 명령 묶음 | 미리 만들어진 모듈 |
| **Runner** | 워크플로 실행 환경 (서버) | 요리할 주방 |

### 구조 시각화

```
.github/workflows/ci.yml  <- Workflow 파일

+---------------------------------------+
|  Workflow: "FDS CI Pipeline"          |
|  +-------------------------------+    |
|  |  Job: test                    |    |
|  |  +-- Step 1: Checkout 코드   |    |
|  |  +-- Step 2: Python 설치     |    |
|  |  +-- Step 3: 의존성 설치     |    |
|  |  +-- Step 4: pytest 실행     |    |
|  +-------------------------------+    |
|  +-------------------------------+    |
|  |  Job: deploy (test 성공 후)   |    |
|  |  +-- Step 1: 서버에 배포     |    |
|  +-------------------------------+    |
+---------------------------------------+
```

In [ ]:
# 예제: GitHub Actions Workflow YAML

yaml_example = """
# .github/workflows/ci.yml
name: FDS CI Pipeline                    # <- Workflow 이름

on:                                       # <- 트리거 조건
  push:
    branches: [main]
  pull_request:
    branches: [main]

jobs:                                     # <- Job 목록
  test:                                   # <- Job 이름
    runs-on: ubuntu-latest               # <- Runner (실행 환경)
    
    steps:                                # <- Step 목록
    - uses: actions/checkout@v5          # <- Action 사용 (2025 표준)
    
    - name: Set up Python                # <- Step 이름
      uses: actions/setup-python@v5      # <- Action 사용 (2025 표준)
      with:
        python-version: '3.11'
    
    - name: Install dependencies         # <- Step 이름
      run: pip install -r requirements.txt  # <- 직접 명령
    
    - name: Run tests                    # <- Step 이름
      run: pytest tests/ --cov=src --cov-report=xml
"""

print(yaml_example)

In [ ]:
# 실습 2: YAML 구조 분석

yaml_analysis = {
    "workflow_name": "FDS CI Pipeline",
    "trigger_events": ["push", "pull_request"],
    "job_count": 1,             # test job 1개
    "runner_os": "ubuntu-latest",
    "step_count": 4,            # checkout, setup-python, install, run tests
}

# 체크포인트
assert yaml_analysis["workflow_name"] == "FDS CI Pipeline"
assert "push" in yaml_analysis["trigger_events"]
assert yaml_analysis["job_count"] == 1
assert yaml_analysis["runner_os"] == "ubuntu-latest"
assert yaml_analysis["step_count"] == 4
print("체크포인트 3 통과! YAML 구조 이해 완료")

### 현업에서 자주 쓰는 Actions (2025 표준)

| Action | 용도 | 버전 |
|--------|------|------|
| `actions/checkout` | 코드 체크아웃 | **v5** |
| `actions/setup-python` | Python 환경 설정 | **v5** |
| `actions/cache` | 의존성 캐싱 (속도 향상) | **v4** |
| `actions/upload-artifact` | 파일 업로드 | **v4** |

> **주의**: 구글링하면 v2, v3 예제가 많지만 2025년 현업 표준은 **v4, v5**입니다!

---
## 4. ML 프로젝트 테스트

### 일반 소프트웨어와 다른 점

```
일반 소프트웨어: "이 함수가 올바른 값을 반환하는가?"

ML 프로젝트: 
   - "이 함수가 올바른 값을 반환하는가?" (기본)
   - "데이터에 문제는 없는가?"
   - "모델 성능이 기준 이상인가?"
   - "추론 속도가 충분히 빠른가?"
```

### ML 테스트 3가지 유형

| 테스트 유형 | 검사 항목 | 실패 시 |
|-------------|----------|--------|
| **데이터 품질** | 결측치, 타입, 범위 | 학습/추론 불가 |
| **모델 성능** | Recall >= 80% 등 | 비즈니스 기준 미달 |
| **추론 시간** | < 100ms | SLA 위반 |

In [ ]:
# 예제: pytest 테스트 코드 (실제 프로젝트에서 사용)

test_code_data = """
# tests/test_data_quality.py
import pytest
import pandas as pd

def test_data_no_nulls(sample_data: pd.DataFrame):
    '''데이터에 결측치가 없어야 함'''
    null_count = sample_data.isnull().sum().sum()
    assert null_count == 0, f"결측치가 {null_count}개 존재합니다"

def test_data_types(sample_data: pd.DataFrame):
    '''필수 컬럼의 타입이 올바른지 확인'''
    assert sample_data['amount'].dtype == 'float64'
    assert sample_data['is_fraud'].dtype == 'int64'

def test_data_range(sample_data: pd.DataFrame):
    '''값 범위가 유효한지 확인'''
    assert sample_data['amount'].min() >= 0, "금액이 음수입니다"
    assert sample_data['is_fraud'].isin([0, 1]).all(), "라벨이 0/1이 아닙니다"
"""
print("데이터 품질 테스트 코드:")
print(test_code_data)

In [ ]:
# 실습 3: 추론 시간 테스트 함수 작성

# 가상의 모델과 데이터
class DummyModel:
    def predict(self, X):
        time.sleep(0.01)  # 10ms 추론 시간 시뮬레이션
        return np.random.randint(0, 2, size=len(X))

model = DummyModel()
X_test = np.random.randn(100, 10)

def test_inference_time_impl(model, X):
    '''추론 시간이 50ms 미만인지 테스트'''
    # 1. 시작 시간 기록
    start = time.time()
    
    # 2. 추론 실행
    model.predict(X)
    
    # 3. 경과 시간 계산
    elapsed = time.time() - start
    
    # 4. assert로 검증 (50ms = 0.05초)
    assert elapsed < 0.05, f"추론 시간 {elapsed:.3f}s > 50ms"
    
    return elapsed

# 테스트 실행
elapsed = test_inference_time_impl(model, X_test[:1])  # 단일 샘플
print(f"체크포인트 4 통과! 추론 시간: {elapsed*1000:.1f}ms")

---
## 5. FDS 비용 함수

### 왜 비용 함수가 필요한가?

```
FDS에서 오류의 비용이 다름!

FN (False Negative) = 사기를 놓침
   -> 평균 피해액 100만원 손실

FP (False Positive) = 오탐 (정상을 사기로)
   -> 고객 불편 + 검토 비용 5만원

FN이 FP보다 20배 비쌈!
-> 단순 Accuracy보다 비용 기반 최적화 필요
```

### 비용 함수 구현

In [ ]:
# FDS 비용 함수 구현

def calculate_fds_cost(y_true, y_pred, fn_cost=1000000, fp_cost=50000):
    """
    FDS 비용 계산
    
    Parameters:
    -----------
    y_true : array-like
        실제 라벨 (0: 정상, 1: 사기)
    y_pred : array-like
        예측 라벨
    fn_cost : int
        FN 비용 (놓친 사기, 기본 100만원)
    fp_cost : int
        FP 비용 (오탐, 기본 5만원)
    
    Returns:
    --------
    dict : 비용 상세 정보
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Confusion Matrix 계산
    tn = ((y_true == 0) & (y_pred == 0)).sum()
    fp = ((y_true == 0) & (y_pred == 1)).sum()
    fn = ((y_true == 1) & (y_pred == 0)).sum()
    tp = ((y_true == 1) & (y_pred == 1)).sum()
    
    # 비용 계산
    total_fn_cost = fn * fn_cost
    total_fp_cost = fp * fp_cost
    total_cost = total_fn_cost + total_fp_cost
    
    # 절감액 (사기 차단)
    savings = tp * fn_cost
    net_savings = savings - total_cost
    
    return {
        "fn_count": fn,
        "fp_count": fp,
        "fn_cost": total_fn_cost,
        "fp_cost": total_fp_cost,
        "total_cost": total_cost,
        "savings": savings,
        "net_savings": net_savings,
    }

print("FDS 비용 함수 정의 완료!")

In [ ]:
# 비용 함수 테스트

# 가상의 예측 결과 (1000건 거래, 5% 사기)
np.random.seed(42)
y_true_test = np.random.choice([0, 1], size=1000, p=[0.95, 0.05])

# 모델 A: Recall 높음 (사기 잘 잡음, 오탐 많음)
y_pred_a = y_true_test.copy()
# FN 5개, FP 50개 추가
fn_indices = np.where(y_true_test == 1)[0][:5]
y_pred_a[fn_indices] = 0  # 5개 놓침
fp_indices = np.where(y_true_test == 0)[0][:50]
y_pred_a[fp_indices] = 1  # 50개 오탐

# 모델 B: Recall 낮음 (사기 많이 놓침, 오탐 적음)
y_pred_b = y_true_test.copy()
fn_indices_b = np.where(y_true_test == 1)[0][:20]
y_pred_b[fn_indices_b] = 0  # 20개 놓침
fp_indices_b = np.where(y_true_test == 0)[0][:10]
y_pred_b[fp_indices_b] = 1  # 10개 오탐

# 비용 비교
cost_a = calculate_fds_cost(y_true_test, y_pred_a)
cost_b = calculate_fds_cost(y_true_test, y_pred_b)

print("=" * 50)
print("모델 A (Recall 높음, 오탐 많음):")
print(f"  FN: {cost_a['fn_count']}건 x 100만원 = {cost_a['fn_cost']:,}원")
print(f"  FP: {cost_a['fp_count']}건 x 5만원 = {cost_a['fp_cost']:,}원")
print(f"  총 비용: {cost_a['total_cost']:,}원")
print()
print("모델 B (Recall 낮음, 오탐 적음):")
print(f"  FN: {cost_b['fn_count']}건 x 100만원 = {cost_b['fn_cost']:,}원")
print(f"  FP: {cost_b['fp_count']}건 x 5만원 = {cost_b['fp_cost']:,}원")
print(f"  총 비용: {cost_b['total_cost']:,}원")
print()
print("=" * 50)
if cost_a['total_cost'] < cost_b['total_cost']:
    print("결론: 모델 A가 비용 효율적! (Recall 중요)")
else:
    print("결론: 모델 B가 비용 효율적!")

---
## 6. 최종 요약

In [ ]:
print("="*60)
print("  2-S4 완료: 비용 최적화 + CI/CD 기초")
print("="*60)
print()
print("배운 것:")
print()
print("1. CI/CD 필요성")
print("   - 수동 배포의 위험성")
print("   - 자동화로 휴먼 에러 방지")
print()
print("2. CI vs CD")
print("   - CI: 코드 변경 시 자동 테스트")
print("   - CD: 테스트 통과 시 자동 배포")
print()
print("3. GitHub Actions (2025 표준)")
print("   - Workflow > Job > Step > Action")
print("   - checkout@v5, setup-python@v5")
print()
print("4. ML 테스트 3가지")
print("   - 데이터 품질, 모델 성능, 추론 시간")
print()
print("5. FDS 비용 함수")
print("   - FN(100만원) vs FP(5만원)")
print("   - Recall 중요!")
print()
print("="*60)
print("다음: 2-S5 A/B 테스트로!")
print("="*60)

### 학습 체크리스트

| 항목 | 이해도 |
|------|--------|
| CI vs CD 구분 | |
| GitHub Actions 구조 (Workflow/Job/Step) | |
| ML 테스트 3가지 유형 | |
| FDS 비용 함수 | |

### 면접 예상 질문

**1. "CI와 CD의 차이는?"**

답변:
- **CI(Continuous Integration)**: 코드 변경 시 자동으로 테스트 실행. 여러 개발자가 작업한 코드를 자주 통합하고 버그를 빨리 발견.
- **CD(Continuous Deployment)**: 테스트 통과 시 자동으로 배포. 사람 개입 없이 더 자주, 더 안전하게 배포.
- ML 프로젝트에서는 코드뿐 아니라 **데이터 변경, 모델 재학습**도 트리거가 됨.

---

**2. "FDS에서 비용 최적화는 어떻게 했나요?"**

답변:
- FN(놓친 사기)과 FP(오탐)의 비용이 다름
- **FN: 평균 100만원 손실**, FP: 5만원 비용
- 이 비용 함수로 최적 Threshold 도출
- 단순 Accuracy보다 **비용 기반 최적화**가 중요